In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 
from pyspark.sql import SparkSession

In [2]:
spark = ( 
 SparkSession
 .builder
 .master("spark://spark-master:7077")
 .appName('MinIO - Teste')
 .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000')
 .config('spark.hadoop.fs.s3a.access.key','minioadmin')
 .config('spark.hadoop.fs.s3a.secret.key','minioadmin')
 .config('spark.hadoop.fs.s3a.path.style.access','true')
 .config('spark.hadoop.fs.s3a.connection.ssl.enabled','false')
 #.config('spark.hadoop.fs.s3a.impl','org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.3.4')
 .config('spark.jars.packages','io.delta:delta-spark_2.12:3.3.1')
 .getOrCreate()
)

print('Processo finalizado!')

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d4d2ebd-5aac-4466-8730-1848cfdd78f8;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (710ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.3.1/delta-spark_2.12-3.3.1.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.3.1!delta-spark_2.12.jar (1

Processo finalizado!


In [3]:
data = [ 
 ("James","","Smith","36636","M",3000),
 ("Michael","Rose","","40288","M",4000),
 ("Robert","","Williams","42114","M",4000),
 ("Maria","Anne","Jones","39192","F",4000),
 ("Jen","Mary","Brown","","F",-1) 
]

schema = (
 StructType([
  StructField("firstname",StringType(),True),
  StructField("middlename",StringType(),True), 
  StructField("lastname",StringType(),True),
  StructField("id", StringType(), True),
  StructField("gender", StringType(), True),
  StructField("salary", IntegerType(), True)
   ])
)

df = spark.createDataFrame(data=data,schema=schema)
df.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+



In [5]:
df.coalesce(1).write.mode('overwrite').parquet('s3a://land/data.parquet')

25/06/01 22:12:26 ERROR TaskSchedulerImpl: Lost executor 1 on 172.18.0.6: worker lost: Not receiving heartbeat for 60 seconds
25/06/01 22:12:26 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.5: worker lost: Not receiving heartbeat for 60 seconds
